# Визуализация районов Москвы по кафе/ресторанам/барам

Бёрем данные с сайта [афиши](https://www.afisha.ru/msk/restaurants) и далее строим интерактивную карту. Район тем лучше, чем больше в нём находится заведений, чем выше средний рейтинг и чем меньше средняя цена. 


## Парсинг данных
Данный код парсит сайт [афиши](https://www.afisha.ru/msk/restaurants)

In [9]:
import requests
from multiprocessing.dummy import Pool as ThreadPool
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

import geocoder

In [16]:
# Данный код берет данные со страницы списка заведений
def getInfoFromResponse(data, resp, num_resp):
    if resp.status_code != 200:
        print("Error", num_resp)
        return
    if num_resp % 20 == 0:   
        print(resp, num_resp)
    bs = BeautifulSoup(resp.text, 'lxml')
    for item in bs.find_all('div', 'places_info'): 
        find_name = item.find('span', 'places_name')
        if find_name is not None: 
            name = find_name.text.strip()
        else: 
            name = None

        find_address = item.find('span', 'places_address')
        if find_address is not None:     
            address = find_address.text.strip()
        else: 
            address = None

        find_metro = item.find('span', 'places_metro')
        if find_metro is not None:     
            metro = find_metro.text.strip()[3:]
        else: 
            metro = None

        find_rating = item.find('div', 'rating_number')
        if find_rating is not None: 
            rating = item.find('div', 'rating_number').text.strip()
        else:
            rating = None
        
        find_money = item.find('span', "range s-tooltip")
        if find_money is not None:
            money = moneyfrom(find_money.get("data-title"))
        else:
            money = None

        data.append((name, address, metro, rating, money))

# Парсинг данных из строки о цене
def moneyfrom(in_str):
    get_data = [int(s) for s in 
                in_str.replace("-", " ").replace("–", " ").split(" ") if s.isdigit()]
    if len(get_data) == 2:
        if get_data[0] == 700:
            return 2
        if get_data[0] == 1500:
            return 3
    else:
        if get_data[0] == 700:
            return 1
        if get_data[0] == 2500:
            return 4

In [ ]:
# Непосредственно парсинг. Всё сохраняется в список data
data = list()
FIRST_URL = "https://www.afisha.ru/msk/restaurants/restaurant_list/"
first_resp = requests.get(FIRST_URL)
getInfoFromResponse(data, first_resp, 1)

t = ThreadPool(2)
start_page = 2
end_page = 453
r = t.map(lambda x: getInfoFromResponse(data, 
                    requests.get("https://www.afisha.ru/msk/restaurants/restaurant_list/page%d/?view=list" % x), x), 
          range(start_page, end_page))
t.close()
t.join()

<Response [200]> 60
<Response [200]> 20
<Response [200]> 80
<Response [200]> 40
<Response [200]> 100
<Response [200]> 120
<Response [200]> 180
<Response [200]> 140
<Response [200]> 200
<Response [200]> 160
<Response [200]> 220
<Response [200]> 240
<Response [200]> 300
<Response [200]> 260
<Response [200]> 320
<Response [200]> 280
<Response [200]> 340
<Response [200]> 360
<Response [200]> 420
Error 430
Error 431
Error 432
Error 433
Error 434
Error 435
Error 436


In [2]:
# Сохраняем данные
df = pd.DataFrame(np.array(data, dtype = object), columns=["Name", "Address", "Metro", "Rating", "Money"])
df.to_csv("parsed_afisha.csv", sep=',', encoding='utf-8')

## Preprocessing
Основная цель данного раздела -- сопоставить каждому заведению район, в котором оно находится. 

In [5]:
df = pd.DataFrame.from_csv("parsed_afisha.csv", sep=',', encoding='utf-8')

In [7]:
# Посмотрим на данные
df.head(4)

,Name,Address,Metro,Rating,Money
0,Хлеб и вино,"просп. Вернадского, 94, корп. 8, ЖК «Миракс-парк»","Юго-Западная, Тропарево",NaN,2.0
1,Beer Happens,"Сретенка, 24/2, стр. 1",Сухаревская,9.1,2.0
2,Винный базар на Комсомольском,"Комсомольский просп., 14/1, корп. 2",Парк культуры,9.1,2.0
3,Grammy's,"Кутузовский просп., 2/1, стр. 6, в Конгресс-па...",Киевская,9.0,3.0


Преобразовывать адрес в координаты будем с помощью библиотеки geocoder, которая подключается к API Yandex для получения всех необходимых данных

Подробности в 
http://geocoder.readthedocs.io/providers/Yandex.html

In [11]:
import geocoder

# Рассмотрим для примера
g = geocoder.yandex('Комсомольский просп., 14/1, корп. 2')
g.json

{'accuracy': 'exact',
 'address': 'Russia, Moscow, Komsomolsky Avenue, 14/1к2',
 'country': 'Russia',
 'country_code': 'RU',
 'description': 'Moscow, Russia',
 'lat': '55.731374',
 'lng': '37.589985',
 'ok': True,
 'quality': 'house',
 'raw': {'Point': {'pos': '37.589985 55.731374'},
  'boundedBy': {'Envelope': {'lowerCorner': '37.58588 55.729057',
    'upperCorner': '37.59409 55.73369'}},
  'description': 'Moscow, Russia',
  'metaDataProperty': {'GeocoderMetaData': {'Address': {'Components': [{'kind': 'country',
       'name': 'Russia'},
      {'kind': 'province', 'name': 'Tsentralny federalny okrug'},
      {'kind': 'province', 'name': 'Moscow'},
      {'kind': 'locality', 'name': 'Moscow'},
      {'kind': 'street', 'name': 'Komsomolsky Avenue'},
      {'kind': 'house', 'name': '14/1к2'}],
     'country_code': 'RU',
     'formatted': 'Moscow, Komsomolsky Avenue, 14/1к2',
     'postal_code': '119034'},
    'AddressDetails': {'Country': {'AddressLine': 'Moscow, Komsomolsky Avenue, 14/1

Приступим непосредственно к извлечению координат

In [ ]:
lat = list()
lng = list()

for address in df.Address:
    if type(address) == str:
        g = geocoder.yandex(address)
        if g.json is not None:
            lat.append(g.json['lat'])
            lng.append(g.json['lng'])
        else:
            lat.append(None)
            lng.append(None)
    else:
        lat.append(None)
        lng.append(None)

In [ ]:
# Добавляем в стоблцы новый датафрейм
df['lng'] = lng
df['lat'] = lat
df['District'] = None

Теперь нам надо подгрузить координаты районов. Мы будем пользоваться библиотекой geopandas (gpd). Координаты районов я взял [отсюда](https://github.com/fall-out-bug/izbirkom_viz/tree/master/atd)

In [83]:
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

# Непосредственно координаты районов
mo_gdf = gpd.read_file('./atd/mo.shp')
mo_gdf.head(7)

,NAME,OKATO,OKTMO,NAME_AO,OKATO_AO,ABBREV_AO,TYPE_MO,geometry
0,Киевский,45298555,45945000,Троицкий,45298000,Троицкий,Поселение,"(POLYGON ((36.8031012 55.4408329, 36.8031903 5..."
1,Филёвский Парк,45268595,45328000,Западный,45268000,ЗАО,Муниципальный округ,"POLYGON ((37.4276499 55.7482092, 37.4284863 55..."
2,Новофёдоровское,45298567,45954000,Троицкий,45298000,Троицкий,Поселение,"POLYGON ((36.8035692 55.4516224, 36.8045117 55..."
3,Роговское,45298575,45956000,Троицкий,45298000,Троицкий,Поселение,"POLYGON ((36.9372397 55.2413907, 36.9372604 55..."
4,"""Мосрентген""",45297568,45953000,Новомосковский,45297000,Новомосковский,Поселение,"POLYGON ((37.4395575 55.6273129, 37.4401803 55..."
5,Вороновское,45298553,45943000,Троицкий,45298000,Троицкий,Поселение,"POLYGON ((36.9700765 55.3548495, 36.9703153 55..."
6,Михайлово-Ярцевское,45298564,45951000,Троицкий,45298000,Троицкий,Поселение,"POLYGON ((37.0527376 55.3947315, 37.0548294 55..."


In [ ]:
# У нас в mo_gdf.geometry есть координаты каждого района
# Данный код итерируется по всем полученным адресам с последующей
# проверкой принадлежности к тому или иному району.

from shapely.geometry import *

gSerDistricts = GeoSeries(mo_gdf.geometry)

for i in range(df.shape[0]):
    if df.iloc[i].lng is not None and df.iloc[i].lat is not None:
        p = Point(float(df.iloc[i].lng), float(df.iloc[i].lat))
        intercept_list = list(gSerDistricts.intersects(p))
        if np.sum(intercept_list) > 0:
            ndf.iloc[i, 7] = mo_gdf[intercept_list].iloc[0].NAME

In [13]:
# Сохраняем данные
df.to_csv("parsed_afisha_districts.csv", sep=',', encoding='utf-8')

In [98]:
df = pd.DataFrame.from_csv("./parsed_afisha_districts.csv", sep=',', encoding='utf-8')

In [99]:
df.head(5)

,Name,Address,Metro,Rating,Money,lat,lng,District
0,Хлеб и вино,"просп. Вернадского, 94, корп. 8, ЖК «Миракс-парк»","Юго-Западная, Тропарево",NaN,2.0,55.652674,37.476402,Тропарёво-Никулино
1,Beer Happens,"Сретенка, 24/2, стр. 1",Сухаревская,9.1,2.0,55.769838,37.632889,Красносельский
2,Винный базар на Комсомольском,"Комсомольский просп., 14/1, корп. 2",Парк культуры,9.1,2.0,55.731374,37.589985,Хамовники
3,Grammy's,"Кутузовский просп., 2/1, стр. 6, в Конгресс-па...",Киевская,9.0,3.0,NaN,NaN,NaN
4,Mitzva Bar,"Пятницкая, 3/4, стр. 1","Новокузнецкая, Третьяковская",9.0,2.0,55.745427,37.627166,Замоскворечье


## Чистка данных
Пусть мы все данные и собирали "руками", но нам, увы, никак не обойти данный раздел. Проблемы следующие: в загруженных данных о "шейпе" Москвы много районов, по которым сайт афиши вообще не имеет никакой информации. Поэтому будет логичным выкинуть из рассмотрения районы, по которым мы ничего сказать не можем (либо можем сказать очень мало), чтобы они не загромождали картину. Плюс к этому, сайт афиши иногда не выдавал всю информацию о том или ином заведении, а geocoder не всегда мог определить адрес и точные координаты. 

### Сначала удалим районы

In [100]:
mask =  (mo_gdf.NAME_AO != "Троицкий") &\
        (mo_gdf.NAME_AO != "Новомосковский") &\
        (mo_gdf.NAME_AO != "Зеленоградский") &\
        (mo_gdf.NAME != "Внуково") &\
        (mo_gdf.NAME != "Восточный")

mo_gdf = mo_gdf[mask]

### Теперь пройдемся по заведениям
Оставим только те заведения, которые есть в районах mo_gdf и по которым есть данные о ценовой категории и выбросим столбец Metro за ненадобностью

In [101]:
mask = list()
for d in df.District:
    if d in list(mo_gdf.NAME):
        mask.append(True)
    else:
        mask.append(False)
df = df[mask]
df = df[df.Money == df.Money]
df = df.drop(['Metro'], axis = 1)

In [102]:
df.count()

Name        7627
Address     7627
Rating      1107
Money       7627
lat         7627
lng         7627
District    7627
dtype: int64

In [ ]:
df.to_csv("./parsed_afisha_cleaned.csv", sep=',', encoding='utf-8')

### Подготовим датасет к отрисовке

In [103]:
df_grouped = df.groupby("District").mean()
df_grouped["Count"] = df.groupby("District").count()["lat"]
df_grouped.head(4)

,Rating,Money,lat,lng,Count
District,,,,,
Академический,6.146154,1.784615,55.684802,37.571744,65
Алексеевский,7.450000,1.536585,55.812362,37.644154,41
Алтуфьевский,NaN,1.428571,55.881580,37.592680,7
Арбат,6.538095,1.820961,55.751379,37.592655,229


In [104]:
mo_gdf_joined = mo_gdf.set_index('NAME').join(df_grouped)
mo_gdf_joined = mo_gdf_joined.reset_index()
mo_gdf_joined.head(4)

,NAME,OKATO,OKTMO,NAME_AO,OKATO_AO,ABBREV_AO,TYPE_MO,geometry,Rating,Money,lat,lng,Count
0,Филёвский Парк,45268595,45328000,Западный,45268000,ЗАО,Муниципальный округ,"POLYGON ((37.4276499 55.7482092, 37.4284863 55...",5.5000,1.510204,55.744490,37.501989,49
1,Щукино,45283587,45372000,Северо-Западный,45283000,СЗАО,Муниципальный округ,"POLYGON ((37.4461022 55.7944941, 37.4461753 55...",6.4875,1.512195,55.801582,37.478297,82
2,Нагатинский Затон,45296573,45919000,Южный,45296000,ЮАО,Муниципальный округ,"POLYGON ((37.6534248 55.65539, 37.6534331 55.6...",7.6000,1.423077,55.678521,37.672042,26
3,Дмитровский,45277574,45339000,Северный,45277000,САО,Муниципальный округ,"POLYGON ((37.4962542 55.8924795, 37.4985112 55...",NaN,1.409091,55.889025,37.533754,22


Введем собственную оценку района

In [105]:
mo_gdf_joined = mo_gdf_joined.fillna(0)
mo_gdf_joined['My_value'] = 0.5 * mo_gdf_joined.Count / max(mo_gdf_joined.Count) +\
                                mo_gdf_joined.Rating / max(mo_gdf_joined.Rating) -\
                                mo_gdf_joined.Money / max(mo_gdf_joined.Money)

Отнормируем все это дело 

In [106]:
mo_gdf_joined['My_value'] = (mo_gdf_joined['My_value'] -\
                                min(mo_gdf_joined['My_value'])) 
mo_gdf_joined['My_value'] /= max(mo_gdf_joined['My_value']) / 10

Теперь у нас рейтинги районов будут от 0 до 10

In [107]:
min(mo_gdf_joined['My_value']), max(mo_gdf_joined['My_value'])

(0.0, 10.0)

## Folium. Интерактивная карта

In [108]:
import folium
import json

Функция окрашивания района в цвет

In [109]:
import matplotlib as mpl
import matplotlib.cm as cm


def dist_color(feature):
    my_value = feature['properties']['My_value']
    
    norm = mpl.colors.Normalize(vmin=min(mo_gdf_joined['My_value']), 
                                vmax=max(mo_gdf_joined['My_value']))
    cmap = cm.hot

    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    color = mpl.colors.to_hex(m.to_rgba(my_value))
    
    return {"fillColor":color, "fillOpacity":0.5,"opacity":0}

Инфо по району

In [110]:
def text(feature):
    dist_name = "<h5>{}</h5>".format(feature['properties']['NAME'])
    money_str = '<br><b>Средняя дороговизна:</b> %.2f' % feature['properties']['Money']
    rating_str = '<br><b>Средний рейтинг:</b> %.2f' % feature['properties']['Rating']
    number_str = '<br><b>Количество:</b> %d' % int(feature['properties']['Count'])
    my_val_str = '<br><b>Наша оценка:</b> %.2f' % feature['properties']['My_value']
    return dist_name + money_str + rating_str + number_str + my_val_str

In [111]:
m = folium.Map(location=[55.764414, 37.647859], zoom_start=9)

for mo in json.loads(mo_gdf_joined.to_json())['features']:
    gj = folium.GeoJson(data=mo, style_function = dist_color, control=False, highlight_function=lambda x:{"fillOpacity":1, "opacity":1}, smooth_factor=0)
    folium.Popup(text(mo)).add_to(gj)
    gj.add_to(m)

Ставим на карту любимые места (в том числе и друзей)

In [112]:
folium.Marker( location=[ 55.759091, 37.624777], popup='Jawsspot', icon=folium.Icon(color='red')).add_to(m)
folium.Marker( location=[ 55.758615, 37.639024], popup='Lora Craft', icon=folium.Icon(color='purple')).add_to(m)
folium.Marker( location=[ 55.765605, 37.609299], popup='Под Мухой', icon=folium.Icon(color='purple')).add_to(m)
folium.Marker( location=[ 55.763822, 37.607359], popup='Porkys', icon=folium.Icon(color='purple')).add_to(m)
folium.Marker( location=[ 55.769388, 37.580795], popup='Практика by Darvin', icon=folium.Icon(color='purple')).add_to(m)
folium.Marker( location=[ 55.759471, 37.645537], popup='Культура', icon=folium.Icon(color='green')).add_to(m)
folium.Marker( location=[ 55.759973, 37.622648], popup='Squat 3/4', icon=folium.Icon(color='green')).add_to(m)
folium.Marker( location=[ 55.760145, 37.646822], popup='Dissident', icon=folium.Icon(color='green')).add_to(m)
folium.Marker( location=[ 55.758509, 37.664384], popup='Arma', icon=folium.Icon(color='green')).add_to(m)
folium.Marker( location=[ 55.692008, 37.530939], popup='Craft Brothers', icon=folium.Icon(color='red')).add_to(m)
folium.Marker( location=[ 55.766101, 37.640075], popup='Волчья Стая', icon=folium.Icon(color='blue')).add_to(m)
folium.Marker( location=[ 55.754897, 37.634560], popup='Китайский лётчик Джао Да', icon=folium.Icon(color='blue')).add_to(m)
folium.Marker( location=[ 55.757977, 37.627184], popup='Разведка', icon=folium.Icon(color='blue')).add_to(m)

In [113]:
m

In [114]:
m.save('./map.html')